In [ ]:
import requests
import re, time
import torch, torchvision
from torch import nn, optim
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!unzip '/content/drive/MyDrive/Fru_Veg_M1.zip' 

In [ ]:
xform = transforms.Compose([transforms.Resize((100,100)), transforms.ToTensor()])
dataset_test = datasets.ImageFolder('/content/Fru_Veg_M1/test', transform=xform)
dataset_train = datasets.ImageFolder('/content/Fru_Veg_M1/train', transform=xform)

In [ ]:
loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=4, shuffle=True)
loader_test = torch.utils.data.DataLoader(dataset_test, batch_size=4, shuffle=True)

model = models.mobilenet_v2(pretrained=True)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


In [ ]:
device = torch.device('cuda:0')
model = model.to(device)

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
criterion = nn.CrossEntropyLoss()

In [ ]:
def run_test(model):
    nsamples_test = len(dataset_test)
    loss, correct = 0, 0
    model.eval()
    with torch.no_grad():
        for samples, labels in loader_test:
            samples = samples.to(device)
            labels = labels.to(device)
            outs = model(samples)
            loss += criterion(outs, labels)
            _, preds = torch.max(outs.detach(), 1)
            correct_mask = preds == labels
            correct += correct_mask.sum(0).item()
    return loss / nsamples_test, correct / nsamples_test

def run_train(model, opt, sched):
    nsamples_train = len(dataset_train)
    loss_sofar, correct_sofar = 0, 0
    model.train()
    with torch.enable_grad():
        for samples, labels in loader_train:
            samples = samples.to(device)
            labels = labels.to(device)
            opt.zero_grad()
            outs = model(samples)
            _, preds = torch.max(outs.detach(), 1)
            loss = criterion(outs, labels)
            loss.backward()
            opt.step()
            loss_sofar += loss.item() * samples.size(0)
            correct_sofar += torch.sum(preds == labels.detach())
    sched.step()
    return loss_sofar / nsamples_train, correct_sofar / nsamples_train

def run_all(model, optimizer, scheduler, n_epochs):
    for epoch in range(n_epochs):
        loss_train, acc_train = run_train(model, optimizer, scheduler)
        loss_test, acc_test = run_test(model)
        print(f"epoch {epoch}: train loss {loss_train:.4f} acc {acc_train:.4f}, test loss {loss_test:.4f} acc {acc_test:.4f}")

In [ ]:
run_all(model, optimizer, scheduler, 5)

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))
/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


epoch 0: train loss 1.4163 acc 0.6531, test loss 0.1314 acc 0.8681
epoch 1: train loss 0.5813 acc 0.8420, test loss 0.0992 acc 0.9022
epoch 2: train loss 0.4219 acc 0.8841, test loss 0.0791 acc 0.9199
epoch 3: train loss 0.3558 acc 0.8996, test loss 0.0734 acc 0.9268
epoch 4: train loss 0.2919 acc 0.9162, test loss 0.0772 acc 0.9297


In [ ]:
!git clone https://github.com/ostrolucky/Bulk-Bing-Image-downloader.git

%run /content/Bulk-Bing-Image-downloader/bbid.py -s "apricot" -o "datasets/apricot" --limit '100'
%run /content/Bulk-Bing-Image-downloader/bbid.py -s "avocado" -o "datasets/avocado" --limit '100'
%run /content/Bulk-Bing-Image-downloader/bbid.py -s "blueberry" -o "datasets/blueberry" --limit '100'
%run /content/Bulk-Bing-Image-downloader/bbid.py -s "cactus fruit" -o "datasets/cactusfruit" --limit '100'
%run /content/Bulk-Bing-Image-downloader/bbid.py -s "Carambula" -o "datasets/Carambula" --limit '100'
%run /content/Bulk-Bing-Image-downloader/bbid.py -s "Chestnut" -o "datasets/Chestnut" --limit '100'
%run /content/Bulk-Bing-Image-downloader/bbid.py -s "Clementine" -o "datasets/Clementine" --limit '100'
%run /content/Bulk-Bing-Image-downloader/bbid.py -s "Coconut" -o "datasets/Coconut" --limit '100'
%run /content/Bulk-Bing-Image-downloader/bbid.py -s "Corn" -o "datasets/Corn" --limit '100'
%run /content/Bulk-Bing-Image-downloader/bbid.py -s "Corn Husk" -o "datasets/CornHusk" --limit '100'
%run /content/Bulk-Bing-Image-downloader/bbid.py -s "Dates" -o "datasets/Dates" --limit '100' 
%run /content/Bulk-Bing-Image-downloader/bbid.py -s "Dragonfruit" -o "datasets/Dragonfruit" --limit '100'
%run /content/Bulk-Bing-Image-downloader/bbid.py -s "Fig" -o "datasets/Fig" --limit '100'
%run /content/Bulk-Bing-Image-downloader/bbid.py -s "Granadilla" -o "datasets/Granadilla" --limit '100'
%run /content/Bulk-Bing-Image-downloader/bbid.py -s "Grapefruit" -o "datasets/Grapefruit" --limit '100'
%run /content/Bulk-Bing-Image-downloader/bbid.py -s "Hazelnut" -o "datasets/Hazelnut" --limit '100'
%run /content/Bulk-Bing-Image-downloader/bbid.py -s "Huckleberry" -o "datasets/Huckleberry" --limit '100'
%run /content/Bulk-Bing-Image-downloader/bbid.py -s "Kaki" -o "datasets/Kaki" --limit '100'
%run /content/Bulk-Bing-Image-downloader/bbid.py -s "Kohlrabi" -o "datasets/Kohlrabi" --limit '100'
%run /content/Bulk-Bing-Image-downloader/bbid.py -s "Kumquats" -o "datasets/Kumquats" --limit '100'
%run /content/Bulk-Bing-Image-downloader/bbid.py -s "Limes" -o "datasets/Limes" --limit '100'
%run /content/Bulk-Bing-Image-downloader/bbid.py -s "Lychee" -o "datasets/Lychee" --limit '100'
%run /content/Bulk-Bing-Image-downloader/bbid.py -s "Mandarine" -o "datasets/Mandarine" --limit '100'
%run /content/Bulk-Bing-Image-downloader/bbid.py -s "Mangostan" -o "datasets/Mangostan" --limit '100'
%run /content/Bulk-Bing-Image-downloader/bbid.py -s "Maracuja" -o "datasets/Maracuja" --limit '100'
%run /content/Bulk-Bing-Image-downloader/bbid.py -s "Melon Piel de Sapo" -o "datasets/Melon_Piel_de_Sapo" --limit '100'
%run /content/Bulk-Bing-Image-downloader/bbid.py -s "Mulberry" -o "datasets/Mulberry" --limit '100'
%run /content/Bulk-Bing-Image-downloader/bbid.py -s "Nectarine" -o "datasets/Nectarine" --limit '100'
%run /content/Bulk-Bing-Image-downloader/bbid.py -s "Papaya" -o "datasets/Papaya" --limit '100'
%run /content/Bulk-Bing-Image-downloader/bbid.py -s "Passion Fruit" -o "datasets/Passion_Fruit" --limit '100'
%run /content/Bulk-Bing-Image-downloader/bbid.py -s "Pepino Fruit" -o "datasets/Pepino" --limit '100'
%run /content/Bulk-Bing-Image-downloader/bbid.py -s "Physalis" -o "datasets/Physalis" --limit '100'
%run /content/Bulk-Bing-Image-downloader/bbid.py -s "Tamarillo" -o "datasets/Tamarillo" --limit '100'
%run /content/Bulk-Bing-Image-downloader/bbid.py -s "Walnut" -o "datasets/Walnut" --limit '100'

!zip -r /content/datasets.zip /content/datasets

In [ ]:
# plots training and validation accuracy on an accuracy vs epochs graph, useful for improving model
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()